# 05 - PennyLane Reference API (Small-Qubit Mode)

Use this when you want an exact/reference result for debugging.

API behavior:
- `pennylane_reference(program, thetas)` -> expvals
- `pennylane_reference(sampler, thetas, shots=...)` -> bitstring samples

Scope: intentionally small-qubit (default <=20) for manageable exact simulation cost.


In [1]:
import os
import sys

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..')) if os.path.basename(os.getcwd()) == 'Tutorial' else os.getcwd()
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

import torch
from src.pauli_surrogate_python import PauliRotation, CliffordGate, PauliSum
from src_tensor.api import (
    compile_expval_program,
    build_quasi_sampler,
    pennylane_reference,
)

In [ ]:
# Small circuit and observables
n_qubits = 4
circuit = [
    CliffordGate('H', [0]),
    CliffordGate('CNOT', [0, 1]),
    PauliRotation('ZZ', [1, 2], param_idx=0),
    PauliRotation('XX', [2, 3], param_idx=1),
]

observables = []
for q in range(n_qubits):
    obs = PauliSum(n_qubits)
    obs.add_from_str('Z', 1.0, qubits=[q])
    observables.append(obs)

thetas = torch.tensor([0.3, -0.4], dtype=torch.float64)

In [ ]:
# Compile tensor expval program
program = compile_expval_program(
    circuit=circuit,
    observables=observables,
    preset='gpu_min',
    preset_overrides={
        'build_device': 'cpu',
        'step_device': 'cpu',
        'stream_device': 'cpu',
        'dtype': 'float64',
        'max_weight': 1_000_000_000,
        'max_xy': 1_000_000_000,
        'offload_steps': False,
        'offload_back': False,
    },
)

tensor_expvals = program.expvals(thetas, stream_device='cpu', offload_back=False)
pl_expvals_via_method = program.expvals_pennylane(thetas)
pl_expvals_via_unified = pennylane_reference(program, thetas)

print('tensor expvals       :', tensor_expvals)
print('pennylane (method)  :', pl_expvals_via_method)
print('pennylane (unified) :', pl_expvals_via_unified)
print('max abs diff tensor-vs-pl:', float(torch.max(torch.abs(tensor_expvals.cpu() - pl_expvals_via_method))))

propagate: 100%|██████████| 4/4 [00:00<00:00, 209.98it/s]


tensor expvals       : tensor([0.0000, 0.0000, 0.9211, 0.9211], dtype=torch.float64)
pennylane (method)  : tensor([0.0000, 0.0000, 0.9211, 0.9211], dtype=torch.float64)
pennylane (unified) : tensor([0.0000, 0.0000, 0.9211, 0.9211], dtype=torch.float64)
max abs diff tensor-vs-pl: 1.2212453270876722e-15


/home/quantum/anaconda3/envs/cudaq/lib/python3.11/site-packages/pennylane/__init__.py:212: PennyLaneDeprecationWarning: PennyLane v0.44 has dropped maintainence support for NumPy < 2.0.0. You have version 1.26.4 installed. Future versions of PennyLane will not work with NumPy<2.0. Please consider upgrading NumPy using `python -m pip install numpy --upgrade`. 
  warnings.warn(


In [ ]:
# Build sampler and pull PennyLane samples from the same circuit
z_combos = [[i] for i in range(n_qubits)]
sampler = build_quasi_sampler(
    n_qubits=n_qubits,
    circuit=circuit,
    z_combos=z_combos,
    max_order=1,
    preset='gpu_min',
    preset_overrides={
        'build_device': 'cpu',
        'step_device': 'cpu',
        'stream_device': 'cpu',
        'dtype': 'float64',
        'max_weight': 1_000_000_000,
        'max_xy': 1_000_000_000,
        'offload_steps': False,
    },
)

samples = pennylane_reference(sampler, thetas, shots=16, seed=0)
print('samples shape:', tuple(samples.shape))
print(samples)

propagate: 100%|██████████| 4/4 [00:00<00:00, 162.34it/s]

zero_filter_backprop: K 4->2 (0.500), n_out 8->2 (0.250), dt=0.108s
samples shape: (16, 4)
tensor([[1, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 0, 0]], dtype=torch.uint8)
